In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

In [ ]:
def init():
    test_df = pd.read_csv("../input/titanic/test.csv")
    train_df = pd.read_csv("../input/titanic/train.csv")
    test_df["Survived"]=-8888
    Main_df=pd.concat((train_df, test_df), axis=0, sort=False)
    print("Checking null values")
    print(Main_df.isnull().sum())
    global title_map
    title_map={
    "mr": "Mr","mrs": "Mrs","miss": "Miss","master": "Master","don": "Royal", "rev": "Officer", "dr": "Officer",
    "mme": "Mrs", "dona": "Royal", "jonkheer" : "Royal", "the countess": "Royal", "capt": "Officer", "col": "Officer",
    "ms": "Mrs", "mlle": "Miss", "major": "Officer", "lady" : "Royal", "sir" : "Royal"
    }
    return(Main_df)

In [ ]:
def get_title(x):
    x=x.split(",")[1].split(".")[0].strip().lower()
    return(title_map[x])

def get_dups(df):
    df["Dup"]= df.duplicated("Name")
    uniq_name=df.loc[df["Dup"]==True, "Name"].unique()
    df["Is_Dup"]=np.where(df["Name"].isin(uniq_name), 1, 0)
    df.drop("Dup", axis=1, inplace=True)
    return(df)  

def is_in_group(df):
    gdf=pd.DataFrame({"GroupSize": df.groupby("Ticket")["Ticket"].count()}).reset_index()
    df=pd.merge(df, gdf, how="left", on="Ticket").set_index(df.index)
    df["NoFamilyGroup"]=np.where((df["SibSp"]==0) & (df["Parch"]==0) & (df["GroupSize"]>1), 1, 0)
    df["Fare"].fillna(8.05, inplace=True)
    df["ActualFare"]=df["Fare"].astype(int)/df["GroupSize"]
    df["IsAlone"]=np.where(df["GroupSize"] ==1, 1, 0)
    
    return(df)

def adjust_columns(df):
    columns=["AgeState", "Title", "Sex", "Embarked", "Deck", "FareBin", "Pclass", "AgeState"]
    df=pd.get_dummies(df, columns=columns)
    df.drop(["Name", "Ticket", "Cabin", "Fare", "Age", "ActualFare", "FamilySize", "NoFamilyGroup"], axis=1, inplace=True)
    return(df)

In [ ]:
if __name__=="__main__":
    df=init() # Init
    
    df["Title"]=df["Name"].map(get_title) #Getting title for each person
    df=get_dups(df) # Duplicates in name
    df=is_in_group(df) # Find group based on tickets #.
    #df["Deck"]= df["Cabin"].apply(lambda x: np.where(pd.notnull(x), str(x)[0].upper(), "Z")) # Deck 
    df["Deck"]= np.where(df["Cabin"].isnull(), 0, 1)
    df["FamilySize"]= df["SibSp"] + df["Parch"] + 1
    
    df["SmallFamily"]= np.where((df["FamilySize"] > 1) & (df["FamilySize"] < 5), 1, 0)
    df["MediumFamily"]= np.where((df["FamilySize"] > 4) & (df["FamilySize"] < 7), 1, 0)
    df["LargeFamily"]= np.where(df["FamilySize"] > 6, 1, 0)


    df["Age"].fillna(df.groupby(['Sex','Title'])["Age"].transform("median"), inplace=True) # Fill age based on median 
    df["AgeState"]= pd.qcut(df["Age"].astype(float), 5, labels=[1, 2, 3,4,5]) # Age buckets
    df["FareBin"]=  pd.qcut(df["ActualFare"], 3, labels=[1,2,3]) # Fare Buckets
    df["FareBin"]= np.where(df["ActualFare"]==0,0,df["FareBin"]) # there are passangers with zero fare
    df["IsMother"]= np.where((df["Sex"]=="female") & (df["Parch"] > 0) & (df["Title"] != "Miss"), 1, 0) # Mothers
    
    # get the value for null embarkment points.
    Embark_imputer=df.groupby(["Pclass", "Embarked"]).median()["ActualFare"].reset_index()
    print(Embark_imputer)
    print(df.loc[df["Embarked"].isna(), ["Pclass", "ActualFare"]])
    df["Embarked"].fillna("C", inplace=True)
    print(df.isnull().sum()) # Now check null columns 

    ##Get the survival rate of key features
    sex_survival_rate=df.loc[df["Survived"]!=-8888, ["Sex", "Survived"]].groupby("Sex").mean().reset_index()
    pclass_survival_rate=df.loc[df["Survived"]!=-8888, ["Pclass", "Survived"]].groupby("Pclass").mean().reset_index()
    embarked_survival_rate=df.loc[df["Survived"]!=-8888, ["Embarked", "Survived"]].groupby("Embarked").mean().reset_index()
    Age_survival_rate=df.loc[df["Survived"]!=-8888, ["AgeState", "Survived"]].groupby("AgeState").mean().reset_index()
    Fare_survival_rate=df.loc[df["Survived"]!=-8888, ["FareBin", "Survived"]].groupby("FareBin").mean().reset_index()
    Deck_survival_rate=df.loc[df["Survived"]!=-8888, ["Deck", "Survived"]].groupby("Deck").mean().reset_index()
    Group_survival_rate=df.loc[df["Survived"]!=-8888, ["NoFamilyGroup", "Survived"]].groupby("NoFamilyGroup").mean().reset_index()
    Alone_survival_rate=df.loc[df["Survived"]!=-8888, ["IsAlone", "Survived"]].groupby("IsAlone").mean().reset_index()
    Mother_survival_rate=df.loc[df["Survived"]!=-8888, ["IsMother", "Survived"]].groupby("IsMother").mean().reset_index()
    SmallF_survival_rate=df.loc[df["Survived"]!=-8888, ["SmallFamily", "Survived"]].groupby("SmallFamily").mean().reset_index()
    MediumF_survival_rate=df.loc[df["Survived"]!=-8888, ["MediumFamily", "Survived"]].groupby("MediumFamily").mean().reset_index()
    LargeF_survival_rate=df.loc[df["Survived"]!=-8888, ["LargeFamily", "Survived"]].groupby("LargeFamily").mean().reset_index()


    
    fig, (axis1, axis2, axis3, axis4, axis5, axis6, axis7, axis8, axis9, axis10, axis11, axis12) = plt.subplots(1,12, figsize=(15,5))
    sn.barplot(x='Sex', y='Survived', data=sex_survival_rate, ax=axis1)
    sn.barplot(x='Pclass', y='Survived', data=pclass_survival_rate, ax=axis2)
    sn.barplot(x='Embarked', y='Survived', data=embarked_survival_rate, ax=axis3)
    sn.barplot(x='AgeState', y='Survived', data=Age_survival_rate, ax=axis4)   
    sn.barplot(x='FareBin', y='Survived', data=Fare_survival_rate, ax=axis5)  
    sn.barplot(x='Deck', y='Survived', data=Deck_survival_rate, ax=axis6) 
    sn.barplot(x='NoFamilyGroup', y='Survived', data=Group_survival_rate, ax=axis7)    
    sn.barplot(x='IsAlone', y='Survived', data=Alone_survival_rate, ax=axis8)    
    sn.barplot(x='IsMother', y='Survived', data=Mother_survival_rate, ax=axis9)    
    sn.barplot(x='SmallFamily', y='Survived', data=SmallF_survival_rate, ax=axis10)    
    sn.barplot(x='MediumFamily', y='Survived', data=MediumF_survival_rate, ax=axis11)    
    sn.barplot(x='LargeFamily', y='Survived', data=LargeF_survival_rate, ax=axis12)    


     
    
    
    df=adjust_columns(df) # Feature encoding and selections.

In [ ]:
#Now model
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score, KFold, learning_curve
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC


In [ ]:
train=df.loc[df["Survived"]!=-8888]
test=df.loc[df["Survived"]==-8888]

X_train=train.drop("Survived", axis=1).values
Y_train=train["Survived"].values

X_test=test.drop("Survived", axis=1).values
print(df.shape, X_train.shape, Y_train.shape, X_test.shape)

In [ ]:
#Let's use cross validation to find best model
kfold=KFold(n_splits=5)

clf=["RandomForest", "LogisticRegression", "SVC"]

random_state = 2
classifiers = []
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(SVC(random_state=random_state))

results, means, std = [], [], []
for classifier in classifiers :
    results.append(cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

for result in results:
    means.append(result.mean())
    std.append(result.std())

result_DF=pd.DataFrame({"Classifiers":clf, "ResultMeans":means, "ResultSTD":std })
print(result_DF)

In [ ]:
def grid_search_param(algo):
    if algo == "Logistic":
        clf=LogisticRegression()
        c_space = np.logspace(-5, 8, 15) 
        penalty=["l1", "l2"]
        param_grid = {'C': c_space, 'penalty' : penalty}
    elif algo=="RandomForest":
        clf=RandomForestClassifier()
        param_grid = { 'n_estimators': [500],
                      'max_features': ['auto', 'sqrt', 'log2'],
                      'max_depth' : [6],
                      'criterion' :['gini', 'entropy']}  

    return(param_grid, clf)

def get_best_param(clf, param):
    grid = GridSearchCV(clf, param, cv = 10) 
    grid.fit(X_train, Y_train) 
    print("Tuned   Parameters: {}".format(grid.best_params_))  
    print("Best score is {}".format(grid.best_score_)) 
    return(grid)


def run_clf(grid, clf, algo):
    print("Hello")
    if algo == "Logistic":
        c_value=grid.best_params_["C"]
        penalty=grid.best_params_["penalty"]
        print("Best parameter: C is {} and Penalty is {}".format(c_value, penalty))
        clf=LogisticRegression(C=c_value, penalty=penalty)
    
    elif algo=="RandomForest":
        n_estimators=grid.best_params_["n_estimators"]
        max_features=grid.best_params_["max_features"]
        max_depth=grid.best_params_["max_depth"]
        criterion=grid.best_params_["criterion"]
        clf=RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, criterion=criterion)
        
    clf.fit(X_train, Y_train)
    print(confusion_matrix(Y_train, clf.predict(X_train)))
    print("{} score is {}".format(algo, clf.score(X_train,Y_train)))
    return(clf)

        

In [ ]:

print("Running Logistic regression")
param, clf=grid_search_param("Logistic")
(grid)= get_best_param(clf, param)
clfLR=run_clf(grid, clf, "Logistic")
pred_values=clfLR.predict(X_test)
test["Survived"]=pd.Series(pred_values, index=test.index)
test[["PassengerId", "Survived"]].to_csv("LRPrediction.csv", index=False)

print("Running Random Forest")
param, clf=grid_search_param("RandomForest")
(grid)= get_best_param(clf, param)
clfRF=run_clf(grid, clf, "RandomForest")  

pred_values=clfRF.predict(X_test)
test["Survived"]=pd.Series(pred_values, index=test.index)
test[["PassengerId", "Survived"]].to_csv("RFPrediction.csv", index=False)